### Automated Hyperparameter Tunning
    -> Bayyesian Optimization
    -> Gradient Descent 
    -> Evolutionary Algorithms
### Bayesian Optimization

    Bayesian approaches, in contrast to random or grid search, keep track of past evaluation results which they 
    use to form a probabilistic model mapping hyperparameters to a probability of a score on the objective 
    function.
                        p(score/hyperparameters)
                        
    Bayesian optimization uses probability to find the minimum of a function.The final aim is to find the input
    value to a function which can give us the lowest possible output value.it usually performs better than 
    random,grid and manual search providing betters performance in the testing phase and reduced optimization
    time.In otimization can be implemented giving three main parameters to the function fmin.
        
            1. Objective function = defines the loss function to minimize.
            2. Domain space = defines the range of input values to test (in bayesian optimization this space
            creates a probability distribution for each of the used hyperparameters)
            3. Optimization Algritham = defines the search algorithm to use to select the best input values to 
            use in each iteration
            
   

In [3]:
import pandas as pd 
import numpy as np

In [4]:
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35,30.5,33.6,0.627,50,1
1,1,85.0,66,29,30.5,26.6,0.351,31,0
2,8,183.0,64,0,30.5,23.3,0.672,32,1
3,1,89.0,66,23,94.0,28.1,0.167,21,0
4,0,137.0,40,35,168.0,43.1,2.288,33,1


In [7]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split
